Final Project 2


* Student name: Reuben Hough
* Student pace: full time
* Scheduled project review date/time: 7/20/20
* Instructor name: Ahbineet
* Blog post URL:


Data Science King County Housing Price

In this project, I'm going to get the data, clean it, get useful information from it, model the data with linear regression, then interpret the results so they will be useful. The first step is importing libraries that I will use.

## Importing libraries

In [ ]:
import sqlite3
import pandas as pd
from pandas.plotting import scatter_matrix
import pickle
import numpy as np
import matplotlib.pyplot
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
from glob import glob
import os
import seaborn as sns
import warnings
from scipy import stats
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as smf
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder, Normalizer, MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, train_test_split, cross_validate
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error, confusion_matrix, make_scorer, auc, accuracy_score, roc_curve
from sklearn.svm import LinearSVC, SVR
from itertools import combinations
%matplotlib inline

## Opening and reading data

In [ ]:
df = pd.read_csv('kc_house_data.csv')
df.head(10)

# Data Observation and Editing

There are clearly null values in the data that need to be dealt with before any kind of assesment is done, but first I will find out what they represent.

Null values exist in the "waterfront", "yr_renovated" and "view" categories, with "?" appearing in the data for "sq_basement"

In [ ]:
print(df['waterfront'].value_counts(dropna=False))

Waterfront refers to whether the property is found on the waterfront "1" or not, "0". With the vast majority of properties not on the waterfront, I believe it is warranted to replace null values with zero.

In [ ]:
print(df['yr_renovated'].value_counts(dropna=False))

It would be inappropriate to replace the null values in the year renovated category with any value other than zero. Since again the vast majority of houses have not been renovated (Indicated by zero), I beleive it would be appropriate to replace null values with zero.

## Dealing with null values

In [ ]:
# Null values in the waterfront and yr_renovated columns are replaced with zero as stated
df.waterfront.replace(np.NaN, 0.0, inplace=True)
df.yr_renovated.replace(np.NaN, 0.0, inplace=True)

In [ ]:
# yr_renovated is turned into integer value
df.yr_renovated = df.yr_renovated.apply(lambda x: int(x))

I've found conflicting data on what "view" represents. It may be the number of times a house has been viewed, or an assessment of the view from the house. Either way, I will not be using this because both are somewhat arbitrary, so it will be dropped.

In [ ]:
df.drop(['view'], axis=1, inplace=True)

## Replacing basement values

There are question marks in the data; thankfully it can all be replaced by subtracting the "above" square footage from total so there is no guessing involved.


In [ ]:
df.sqft_basement = (df.sqft_living - df.sqft_above)

## Editing the date

In [ ]:
# Making the Sale Date more useable
df['Year'] = df['date'].map(lambda x: '{}'.format(x[-4:].replace('/', '')))
df['Month'] = df['date'].map(lambda x: '{}'.format(x[:2].replace('/', '')))
df['Day'] = df['date'].map(lambda x: '{}'.format(x[-7:-5].replace('/', '')))
df['Day'] = df['Day'].map(lambda x: int(x))
df['Month'] = df['Month'].map(lambda x: int(x))
df['Year'] = df['Year'].map(lambda x: int(x))

In [ ]:
# Making a function that turns the new date columns into a julian date
def julian(a,b,c):
    w=[]
    for x,y,z in zip(a,b,c):
         w.append(pd.Timestamp(year = x,  month = y, day = z).to_julian_date())
    return w

In [ ]:
# Turning the date into a Julian date to make it more useable
df['Julian'] = julian(df['Year'], df['Month'], df['Day'])
df.sort_values(by='Julian')

Looking at the data for any obvious errors

In [ ]:
print(df.describe())
print(df.info())

33 bedrooms is almost certainly an error based on the square footage; it's probably meant to be 3, but I'm uncertain so it will be dropped

In [ ]:
df = df.drop([15856])

## Checking for duplicate entries

In [ ]:
# There are none
df.duplicated(keep=False).sum()

# Creating Data

## Making a new catergory "Seattle" That contains all zip codes within the greater Seattle area

In [ ]:
# list found on seattlearea.com/zip-codes/
seattle = [98003, 98005, 98033, 98037, 98040, 98052, 98055, 98101, 98101, 98102,
            98103, 98103, 98103, 98104, 98104, 98105, 98105, 98107, 98109,
            98109, 98110, 98110, 98116, 98116, 98118, 98121, 98125, 98144, 98199]

In [ ]:
# Creating category with a 1 if in Seattle and 0 if not
df['Seattle'] = df['zipcode'].apply(lambda x: 1 if x in seattle else 0)

## Binning Month by Season

In [ ]:
# Puts december next to other winter months
forbin = df['Month'].replace(12,0)

In [ ]:
monthbins = [0,3,6,9,11]
seasonlabels = ['Winter', 'Spring', 'Summer', 'Autumn']
binmonth = pd.cut(forbin, monthbins, labels = seasonlabels, include_lowest = True).cat.as_unordered()

In [ ]:
binmonth.value_counts().plot(kind='bar',color='gray')
plt.ylabel('Houses Sold')

In [ ]:
df['season'] = binmonth

Creating "basement" as a category for houses with a basement

In [ ]:
df['basement'] = df['sqft_basement'].astype(bool).astype(int)

# Relationship between size of a house and lot and the size of its neighbors

In [ ]:
fig = plt.figure(figsize = (12,5))

ax1 = fig.add_subplot(121)
ax1.scatter(df['sqft_living15'], df['sqft_living'], alpha = 0.3);
ax1.set_title('Living space in ft^2');
ax1.set_xlabel('15 nearest neighbours area');
ax1.set_ylabel('House area');

ax1 = fig.add_subplot(122)
ax1.scatter(df['sqft_lot15'], df['sqft_lot'], alpha = 0.3);
ax1.set_title('Lot space in ft^2');
ax1.set_xlabel('15 nearest neighbours area');
ax1.set_ylabel('House area');

# Checking for Correlations in the Data to Eliminate Multicollinearity

These are subplots comparing price to several different factors

In [ ]:
variables = ['floors', 'bedrooms', 'bathrooms','waterfront','condition','grade','Month','Seattle','season','basement']
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(14,20))

for column, ax in zip(variables, axes.flatten()):
    (df.groupby(column).mean()['price'].sort_values().plot.bar(ax=ax))                
    
    ax.set_title(column)
    
fig.tight_layout()

Log transforming the price may the data easier to work with

In [ ]:
df['logprice'] = np.log(df['price'])
df[['price', 'logprice']].hist(figsize=(12,5),bins=30,color = "skyblue");

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(14,20))

for column, ax in zip(variables, axes.flatten()):
    (df.groupby(column).mean()['logprice'].sort_values().plot.bar(ax=ax))                
    
    ax.set_title(column)
    
fig.tight_layout()

In [ ]:
variables = ['floors', 'bedrooms', 'bathrooms', 'sqft_living','sqft_lot', 'sqft_above', 'waterfront','condition','grade','sqft_living15','sqft_lot15','Month','Seattle','season','basement']
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df[variables].corr(), cmap='coolwarm', annot=True, linewidths=.5, ax=ax);

Several things are already noticable. There is minor correlation between bathrooms, grade, and bedrooms, which is expected. Living in the Seattle area, having a basement, and having more floors are good indicators of price.

Also sqft_above seems to have strong correlation with multiple columns so it should be eliminated.

In [ ]:
df.drop(['sqft_living'], axis=1, inplace=True)

## Binning month by season

In [ ]:
# Puts december next to other winter months
forbin = df['Month'].replace(12,0)

In [ ]:
monthbins = [0,3,6,9,11]
seasonlabels = ['Winter', 'Spring', 'Summer', 'Fall']
binmonth = pd.cut(forbin, monthbins, labels = seasonlabels, include_lowest = True).cat.as_unordered()

In [ ]:
binmonth.value_counts().plot(kind='bar')

Spring clearly has the most houses purchased, and fall is a very unpopular time to buy a house

## Checking for and removing outliers

Outliers will skew the data, so getting rid of a few outliers should help in that regard

In [ ]:
df.loc[df['sqft_lot15']>500000]

In [ ]:
df.loc[df['price']>6000000]

In [ ]:
df.loc[df['sqft_lot']>1000000]

In [ ]:
df.drop(index=[9509,13177,20010,3846,7105,9059,1687,7489,7610,16940], inplace=True)

## Creating a function to examine a histogram and Q-Q plot, then plotting

In [ ]:
def diagnostic_plot(df, var):
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    df[var].hist(bins=30)
    plt.title(var)
    plt.subplot(1, 2, 2)
    stats.probplot(df[var], dist="norm", plot=plt)
    plt.title(var)
    plt.tight_layout()
    plt.show()

In [ ]:
variables = ['floors', 'bedrooms', 'bathrooms', 'sqft_above','sqft_lot','waterfront','condition','grade','sqft_living15','sqft_lot15','Month','Seattle','basement']
for col in df[variables]:
    diagnostic_plot(df,col)

# Using Linear Regression to Predict Prices

## Dropping columns that are no longer useful

In [ ]:
df.drop(['id', 'date', 'lat', 'long'], axis=1, inplace=True)

## Creating dummy variables from categorical variables

In [ ]:
categoricals = ['condition','grade','zipcode','basement','season','waterfront','Month']

In [ ]:
conditiondum = pd.get_dummies(df["condition"], prefix="condition", drop_first=True)
gradedum = pd.get_dummies(df["grade"], prefix="grade", drop_first=True)
zipcodedum = pd.get_dummies(df["zipcode"], prefix="zipcode", drop_first=True)
basementdum = pd.get_dummies(df["basement"], prefix="basement", drop_first=True)
seasondum = pd.get_dummies(df["season"], prefix="season", drop_first=True)
waterfrontdum = pd.get_dummies(df["waterfront"], prefix="waterfront", drop_first=True)
monthdum = pd.get_dummies(df["Month"], prefix="month", drop_first=True)
olddf = df

df.drop(categoricals, axis=1, inplace = True)
df = pd.concat([df, conditiondum, gradedum, zipcodedum, basementdum, seasondum, waterfrontdum, monthdum], axis=1)

## Looking for multicollinearity

This funtion shows a large amount of subplots

In [ ]:
noprice = df.drop(['price', 'logprice'], axis = 1)

plt.figure(1, figsize=(20,20))
        
def multi_scatter_plot(x):
    sns.scatterplot(x, y="logprice", data=df) #iterating over dataframe minus price columns, adding subplots

for index, col in enumerate(noprice.columns, start=1):

    plt.subplot(11, 11,index)

    multi_scatter_plot(col)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(olddf.corr(), cmap='coolwarm', annot=True, linewidths=.5, ax=ax);

The correlation is not too significant (under .75)

## Transforming data to normalize

Normalizing the data makes it easier to visualize

In [ ]:
df['sqft_above'] = np.log(df['sqft_above'])
df['sqft_above'].plot.hist(density=True )
df['sqft_above'].plot.kde(label='sqft_above')
plt.legend()
plt.show()

In [ ]:
df['sqft_living15'] = np.log(df['sqft_living15'])
df['sqft_living15'].plot.hist(density=True )
df['sqft_living15'].plot.kde(label='sqft_living15')
plt.legend()
plt.show()

In [ ]:
df['sqft_lot'] = np.log(df['sqft_lot'])
df['sqft_lot'].plot.hist(density=True )
df['sqft_lot'].plot.kde(label='sflot_log')
plt.legend()
plt.show()

In [ ]:
df['sqft_lot15'] = np.log(df['sqft_lot15'])
df['sqft_lot15'].plot.hist(density=True )
df['sqft_lot15'].plot.kde(label='sqft_lot15')
plt.legend()
plt.show()

In [ ]:
df['yr_built'] = np.log(df['yr_built'])
df['yr_built'].plot.hist(density=True )
df['yr_built'].plot.kde(label='yr_built')
plt.legend()
plt.show()

# Performing Regression Diagnostics

## Train test split

The train test split is used to validate our regression model

In [ ]:
X = df.drop(['logprice'], axis=1)
y = df['logprice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
x_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model_fit = sm.OLS(y_train, x_train).fit()
results_df = pd.concat([x_train, y_train], axis=1)
model_fit.summary()

In [ ]:
df.columns[1:]

In [ ]:
lr = LinearRegression()

#fitting model on training data
lr.fit(X_train,y_train)

#predicting on the training data
y_pred=lr.predict(X_train)

#calculating score on the linear regression model on the training set
r_score=lr.score(X_train,y_train)
r_score

The r score is very good. I don't believe it's a result of overfitting because I haven't used polynomial regression. It may be due to the way the log of the price normalized so well.

In [ ]:
# waterfront caused an error, possibly related to its name'1.0', so it needs to be dropped
df.drop(['waterfront_1.0'], axis=1, inplace=True)

In [ ]:
# Here I'm getting the rsquared values so that I can decide which variables are the best predictors
col_names = df.describe().columns.drop(['logprice'])
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Housing: Price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = ols(formula=f, data=df).fit()
    X_new = pd.DataFrame({val: [df[val].min(), df[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    # param[0] is slope and param[1] is intercept
    print(results[idx+1])

Bathrooms, sq_above, sq_basement, sqft_living15, grade_10, grade_11, have a relatively high r squared value

Year, Julian, condition_3, grade_8, zipcode_98011, zipcode_98024, zipcode_98034, zipcode_98065, zipcode_98070, zipcode_98072, zipcode_98107, zipcode_98136, season_Summer, season_Autumn, month_3, month_5,month_7,month_8, month_9, month_10, month_11, and month_12 have p values higher than the acceptable .05

In [ ]:
df.drop(['Year', 'Julian', 'condition_3', 'grade_8', 'zipcode_98011', 'zipcode_98024', 'zipcode_98034', 'zipcode_98065', 'zipcode_98070', 'zipcode_98072', 'zipcode_98107', 'zipcode_98136', 'season_Summer', 'season_Autumn', 'month_3', 'month_5','month_7','month_8', 'month_9', 'month_10', 'month_11', 'month_12'], axis=1, inplace=True)

In [ ]:
col_names = df.describe().columns.drop(['logprice'])
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Housing: Price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = ols(formula=f, data=df).fit()
    X_new = pd.DataFrame({val: [df[val].min(), df[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    # param[0] is slope and param[1] is intercept
    print(results[idx+1])

In [ ]:
f = 'logprice ~ {}'.format(df.column)
model = ols(formula=f, data=df).fit()
model.summary()

The rsquared value is far from useful, so I'm going to pick the top variables based on p-values. I believe it would be more broadly useful from a business perspective to include different sorts of variables; in this case the top two months, the top two addition related variables, and the top two zipcodes based on rsquared values.

In [ ]:
new_df = df[['logprice','sqft_above', 'bathrooms', 'zipcode_98004', 'zipcode_98112', 'month_2', 'month_4']]

## Statsmodel regression

In [ ]:
# refitting the model
f = 'logprice ~ sqft_above + bathrooms + zipcode_98004 + zipcode_98112 + month_2 + month_4'
model = ols(formula=f, data=new_df).fit()
model.summary()

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

Checking for normality. Looks very good

In [ ]:
predictions = model_fit.predict(X_test)
plt.scatter(y_test,predictions)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')

# Conclusion

The model was checked for the regression assumptions and appeared to pass. The r_squared value is somewhat low, however it the predictions compared to actual prices look reasonably linear.